<a href="https://colab.research.google.com/github/linjiw/linji85/blob/main/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##log in kaggle -> download dataset -> unzip dataset

In [ ]:
import json

TOKEN = {"username":"xxx","key":"xxx"}

! pip install kaggle==1.5.12
! mkdir -p .kaggle
! mkdir -p /content & mkdir -p /content/.kaggle & mkdir -p /root/.kaggle/

with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(TOKEN, file)

! pip install --upgrade --force-reinstall --no-deps kaggle
! ls "/content/.kaggle"
! chmod 600 /content/.kaggle/kaggle.json
! cp /content/.kaggle/kaggle.json /root/.kaggle/

! kaggle config set -n path -v /content

! kaggle competitions download -c 11-785-s22-hw1p2

! unzip competitions/11-785-s22-hw1p2/11-785-s22-hw1p2.zip


##mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Here I use wand to log my experiment process.

In [ ]:
%%capture
!pip install wandb --upgrade

import wandb

wandb.login()

wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


##import libraries

In [ ]:
import os
import csv
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
from tqdm import tqdm
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##define model architechture

## Model trainning log

### Increase model size
For the model architechture, I tried a lot from scratch. 
Frist I use a simple MLP with just three layers. And that cannot work well. Then after calculated the input layer is about 273, which would a large hidden layer should bring performance increase. Then I started to add layers and layer size. The performace starts increase to 79.

And then I found that I have not use the whole train dataset, that might be a reason why the performance is not good enought. After moving to the whole dataset, the performance has increased to 81.


### Increase context size
Then I started to try other parameters, first I increased my context to 30, and this brought slightly performance increase. Then I did ablation study for the context, and found that context around 25-30 could bring good results.


### Use cylinder architechture
After stay the context to 30, I noticed that students in piazza claims that cylinder architecther usually have better performance, then I replaced my neurons for each layer to 4096 and stay keeped other layers in 2048 which still is a great number. This brought me to around 84. 


### Change dropout value to avoid overfitting
Also, in the process I played with the dropout values and found that small dropout value could increase the training speed but will easily bring my model to overfitting. Thus, I found that I could lower my learning rate before the overfitting happens, and this bring a good results.


### reduce learning rate
However, 84 is not enoough to get full credit. Then I started to turn my learning rate much lower when my model reached to 84. The learning rate I use is 0.0001 and it could keep my model increase to 85, which is a great results. 

### R-Dropout
Beyound these, I also implemented the R-Dropout to calculate the loss, however, since I did not do an abaltion study for that, I cannot report the performance for that.

In [ ]:

class Network(torch.nn.Module):
    def __init__(self,context=30,dropout=0.4):
        super(Network, self).__init__()
        in_size = 13 * (1+2*context)

        self.layers = nn.Sequential(
            nn.Linear(in_features=in_size,out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.BatchNorm1d(num_features=4096),
            nn.Linear(in_features=4096,out_features=4096),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.BatchNorm1d(num_features=4096),
            nn.Linear(in_features=4096,out_features=2048),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.BatchNorm1d(num_features=2048),
            nn.Linear(in_features=2048,out_features=2048),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.BatchNorm1d(num_features=2048),
            nn.Linear(in_features=2048,out_features=1024),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.BatchNorm1d(num_features=1024),
            nn.Linear(in_features=1024,out_features=1024),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.BatchNorm1d(num_features=1024),
            nn.Linear(in_features=1024,out_features=512),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.BatchNorm1d(num_features=512),
            nn.Linear(in_features=512,out_features=40),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.BatchNorm1d(num_features=40),
        )



        # self.laysers = nn.Sequential(*layers)

    def forward(self, A0):
        x = self.layers(A0)
        return x


In [ ]:
Network(10,dropout=0.4)

Network(
  (layers): Sequential(
    (0): Linear(in_features=273, out_features=4096, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, inplace=False)
    (3): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Linear(in_features=4096, out_features=4096, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.4, inplace=False)
    (7): BatchNorm1d(4096, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): Linear(in_features=4096, out_features=2048, bias=True)
    (9): ReLU()
    (10): Dropout(p=0.4, inplace=False)
    (11): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Linear(in_features=2048, out_features=2048, bias=True)
    (13): ReLU()
    (14): Dropout(p=0.4, inplace=False)
    (15): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (16): Linear(in_features=2048, out_features=1024, bias=True)
    (17): ReLU()
    (18): Dropout(p=0.4, inplace=False)


##Dataset function

In [ ]:

class LibriSamples(torch.utils.data.Dataset):
    def __init__(self, data_path, sample=20000, shuffle=True, partition="dev-clean", csvpath=None):
        # sample represent how many npy files will be preloaded for one __getitem__ call
        self.sample = sample 
        
        self.X_dir = data_path + "/" + partition + "/mfcc/"
        self.Y_dir = data_path + "/" + partition +"/transcript/"
        
        self.X_names = os.listdir(self.X_dir)
        self.Y_names = os.listdir(self.Y_dir)

        # using a small part of the dataset to debug
        if csvpath:
            subset = self.parse_csv(csvpath)
            self.X_names = [i for i in self.X_names if i in subset]
            self.Y_names = [i for i in self.Y_names if i in subset]
        
        if shuffle == True:
            XY_names = list(zip(self.X_names, self.Y_names))
            random.shuffle(XY_names)
            self.X_names, self.Y_names = zip(*XY_names)
        
        assert(len(self.X_names) == len(self.Y_names))
        self.length = len(self.X_names)
        
        self.PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
      
    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row[1])
        return subset[1:]

    def __len__(self):
        return int(np.ceil(self.length / self.sample))
        
    def __getitem__(self, i):
        sample_range = range(i*self.sample, min((i+1)*self.sample, self.length))
        
        X, Y = [], []
        for j in sample_range:
            X_path = self.X_dir + self.X_names[j]
            Y_path = self.Y_dir + self.Y_names[j]
            
            label = [self.PHONEMES.index(yy) for yy in np.load(Y_path)][1:-1]

            X_data = np.load(X_path)
            X_data = (X_data - X_data.mean(axis=0))/X_data.std(axis=0)
            X.append(X_data)
            Y.append(np.array(label))
            
        X, Y = np.concatenate(X), np.concatenate(Y)
        return X, Y
    
class LibriItems(torch.utils.data.Dataset):
    def __init__(self, X, Y, context = 0):
        assert(X.shape[0] == Y.shape[0])
        
        self.length  = X.shape[0]
        # print(f"self.length {self.length}")
        self.context = context
        empty_matrix = np.zeros((context,13))
        if context == 0:
            self.X, self.Y = X, Y
        else:
            # TODO: self.X, self.Y = ... 
            self.X = np.concatenate((empty_matrix,X,empty_matrix ))
            self.Y = Y
            pass 
        
    def __len__(self):
        return self.length
        
    def __getitem__(self, i):
        if self.context == 0:
            xx = self.X[i].flatten()
            yy = self.Y[i]
        else:
            # TODO xx, yy = ...
            # xx = self.X[self.context:]
            xx = self.X[i:i+2*self.context+1].flatten()
            # print(f"xx.shape {xx.shape} pos {i}")

            yy = self.Y[i]
            pass
        return xx, yy
    



##train function

In [ ]:
from tqdm import trange
def compute_kl_loss( p, q, pad_mask=None):
    
    p_loss = torch.nn.functional.kl_div(torch.nn.functional.log_softmax(p, dim=-1), torch.nn.functional.softmax(q, dim=-1), reduction='none')
    q_loss = torch.nn.functional.kl_div(torch.nn.functional.log_softmax(q, dim=-1), torch.nn.functional.softmax(p, dim=-1), reduction='none')
    
    # pad_mask is for seq-level tasks
    if pad_mask is not None:
        p_loss.masked_fill_(pad_mask, 0.)
        q_loss.masked_fill_(pad_mask, 0.)

    # You can choose whether to use function "sum" and "mean" depending on your task
    p_loss = p_loss.mean()
    q_loss = q_loss.mean()

    loss = (p_loss + q_loss) / 2
    return loss
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer
def build_network(conext=30, dropout=0.5):
    network = Network(conext,dropout)
    # network.load_state_dict(torch.load("/content/drive/MyDrive/11785/model/model_epoch_acc_81.43418102540598_12_16_02_2022_11_52_06.pth"))
    return network.to(device)


def train(args, model, device, train_samples, optimizer,scheduler, criterion, epoch, batch_size):
    model.train()
    # print(f"train_samples.shape: {train_samples.shape}")
    # print(4)
    
    # scheduler = ExponentialLR(optimizer, gamma=0.9)


    for i in trange(len(train_samples)):
        X, Y = train_samples[i]
        train_items = LibriItems(X, Y, context=args['context'])
        # print(5)
        train_loader = torch.utils.data.DataLoader(train_items, batch_size=batch_size, shuffle=True)
        # print(f"X.shape: {X.shape}")
        for batch_idx, (data, target) in enumerate(train_loader):
            # print(f"batch_idx: {batch_idx} data {data.shape}")
            data = data.float().to(device)
            target = target.long().to(device)

            optimizer.zero_grad()
            
            logits = model(data)
            
            logits2 = model(data)

            # cross entropy loss for classifier
            ce_loss = 0.5 * (criterion(logits, target) + criterion(logits2, target))

            kl_loss = compute_kl_loss(logits, logits2)

            # carefully choose hyper-parameters
            loss = ce_loss + 5 * kl_loss

            # loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            if batch_idx % args['log_interval'] == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_idx * len(data), len(train_loader.dataset),
                  100. * batch_idx / len(train_loader), loss.item()))
    # scheduler.step()
    
    return loss.cpu().detach().numpy()


def test(args, model, device, dev_samples,criterion):
    model.eval()
    true_y_list = []
    pred_y_list = []
    with torch.no_grad():
        for i in trange(len(dev_samples)):
            X, Y = dev_samples[i]

            test_items = LibriItems(X, Y, context=args['context'])
            test_loader = torch.utils.data.DataLoader(test_items, batch_size=args['batch_size'], shuffle=False)

            for data, true_y in test_loader:
                data = data.float().to(device)
                true_y = true_y.long().to(device)                
                
                output = model(data)
                pred_y = torch.argmax(output, axis=1)
                
                loss = criterion(output, true_y)

                pred_y_list.extend(pred_y.tolist())
                true_y_list.extend(true_y.tolist())
    train_accuracy =  accuracy_score(true_y_list, pred_y_list)
    
    return train_accuracy , loss.cpu().detach().numpy()

def savemodel(model,epoch,acc):
    now = datetime.now()
    pth = "/content/drive/MyDrive/11785/model/"
    name = now.strftime("%d_%m_%Y_%H_%M_%S")
    torch.save(model.state_dict(), f"{pth}model_epoch_acc_{acc}_{epoch}_{name}.pth")
    # files.download(f"{pth}model_epoch_{epoch}_{name}.pth")


def main(config=None):
    args = {
        'batch_size': 1024*4,
        'context': 30,
        'log_interval': 1000,
        'LIBRI_PATH': './hw1p2_student_data',
        'lr': 0.001,
        'epoch': 200
    }
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        # print(1)

        # loader = build_dataset(config.batch_size)
        print(f"context {config.context}")
        network = build_network(config.context, config.dropout)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)


        # print(2)
        criterion = torch.nn.CrossEntropyLoss()
        train_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="train-clean-100", csvpath="/content/drive/MyDrive/11785/train_filenames_subset_8192_v2.csv")
        # train_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="train-clean-100", csvpath=None)
        dev_samples = LibriSamples(data_path = args['LIBRI_PATH'], shuffle=True, partition="dev-clean")
        # print(3)
        
        print(config.epochs)
        for epoch in range(1, config.epochs + 1):
            # print(4)
            print(f"now_lr: {scheduler.get_last_lr()}")
            train_loss = train(args, network, device, train_samples, optimizer,scheduler, criterion, epoch,config.batch_size)
            # print(5)
            test_acc, test_loss = test(args, network, device, dev_samples,criterion)
            print('Dev accuracy ', test_acc)
            for param_group in optimizer.param_groups:
                print(f"lr: {param_group['lr']}")
            savemodel(network,epoch,test_acc*100)

            wandb.log({"loss": test_loss, "epoch": epoch})
            scheduler.step()
    

## Wandb Sweep
Wandb sweep could auto sweep the parameters for you and log the results. However, I started to know the wandb after I got a 86 acc, thus, I did not use this sweep so much.

In [ ]:
sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['adam']
        },
    'dropout': {
          'values': [0.1]
        },
    'context': {
          'values': [30]
        },
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'epochs': {
        'value': 20}
    })

import math

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0.08,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'values': [512*16]
      }
    })
import pprint

pprint.pprint(sweep_config)
sweep_id = wandb.sweep(sweep_config, project="pytorch-test1")


{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'values': [8192]},
                'context': {'values': [30]},
                'dropout': {'values': [0.1]},
                'epochs': {'value': 20},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0.08},
                'optimizer': {'values': ['adam']}}}
Create sweep with ID: mrrwt8dv
Sweep URL: https://wandb.ai/linjiw/pytorch-test1/sweeps/mrrwt8dv


In [ ]:
wandb.agent(sweep_id, main, count=10)

wandb: Agent Starting Run: mn1ecptu with config:
wandb: 	batch_size: 8192
wandb: 	context: 30
wandb: 	dropout: 0.1
wandb: 	epochs: 20
wandb: 	learning_rate: 0.08907761174737502
wandb: 	optimizer: adam


cuda


context 30
20
now_lr: [0.08907761174737502]


  0%|          | 0/1 [00:00<?, ?it/s]

Train Epoch: 1 [0/10391263 (0%)]	Loss: 4.676150
Train Epoch: 1 [8192000/10391263 (79%)]	Loss: 1.371432


...

Train Epoch: 17 [0/10391263 (0%)]	Loss: 0.794803
Train Epoch: 17 [8192000/10391263 (79%)]	Loss: 0.811939


100%|██████████| 1/1 [00:43<00:00, 43.35s/it]


Dev accuracy  0.805650179406822
lr: 0.016506261294259188
now_lr: [0.01485563516483327]


  0%|          | 0/1 [00:00<?, ?it/s]

Train Epoch: 18 [0/10391263 (0%)]	Loss: 0.797178


#Test and submit results to kaggle

In [ ]:

class test_sample(torch.utils.data.Dataset):
    def __init__(self, data_path, sample=20000, shuffle=True, partition="", csvpath=None):
        # sample represent how many npy files will be preloaded for one __getitem__ call
        self.sample = sample 
        
        self.X_dir = data_path + "/" + partition + "/mfcc/"
        
        self.X_names = os.listdir(self.X_dir)

        # using a small part of the dataset to debug
        # if csvpath:
        #     # subset = self.parse_csv(csvpath)
        #     # self.X_names = [i for i in self.X_names if i in subset]
        self.X_names = list(pd.read_csv(csvpath).file)
        
        # print(self.X_names)
        
        self.length = len(self.X_names)
        
        self.PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
      
    @staticmethod
    def parse_csv(filepath):
        subset = []
        with open(filepath) as f:
            f_csv = csv.reader(f)
            for row in f_csv:
                subset.append(row[0])
                # print(row[0])
        return subset[0:]

    def __len__(self):
        # return int(np.ceil(self.length / self.sample))
        return len(self.X_names)
        
    def __getitem__(self, i):
        # sample_range = range(i*self.sample, min((i+1)*self.sample, self.length))
        
        X = []
        for j in self.X_names:
            X_path = self.X_dir + j
            
            # print(X_path)
            X_data = np.load(X_path)
            X_data = (X_data - X_data.mean(axis=0))/X_data.std(axis=0)
            X.append(X_data)
            
        X = np.concatenate(X)
        return X
    
class test_item(torch.utils.data.Dataset):
    def __init__(self, X, context = 0):
        
        self.length  = X.shape[0]
        self.context = context

        self.context = context
        empty_matrix = np.zeros((context,13))
        if context == 0:
            self.X = X
        else:
            # TODO: self.X, self.Y = ... 
            self.X = np.concatenate((empty_matrix,X,empty_matrix ))
            pass 
        
    def __len__(self):
        return self.length
        
    def __getitem__(self, i):
        if self.context == 0:
            xx = self.X[i].flatten()
        else:
            # TODO xx, yy = ...
            xx = self.X[i:i+2*self.context+1].flatten()

            pass
        return xx
    



In [ ]:
def submit_test(args, model, device, test_samples):
    model.eval()
    true_y_list = []
    pred_y_list = []
    with torch.no_grad():
        for i in range(1):
            X = test_samples[i]

            test_items = test_item(X, context=args['context'])
            test_loader = torch.utils.data.DataLoader(test_items, batch_size=args['batch_size'], shuffle=False)

            for data in test_loader:
                data = data.float().to(device)
                                
                output = model(data)
                pred_y = torch.argmax(output, axis=1)

                pred_y_list.extend(pred_y.tolist())
    # print(pred_y_list)
    now_name = datetime.now().strftime("%d_%m_%Y_%H_%M_%S")
    f = open(f"bad_{now_name}.csv", "w")
    f.write("id,label\n")
    for idx, i  in enumerate(pred_y_list):
        f.write(f"{idx},{i}\n")
    f.close()

 
    # with open('good.csv', 'w', newline='') as csvfile:
    #     writer = csv.DictWriter(csvfile, fieldnames = ['id','label'])
    #     # writer.writerow(['id','label'])
    #     writer.writeheader() 
    #     for idx, i  in enumerate(pred_y_list):
    #         writer.writerow([idx,i])
        
    
    


In [ ]:
def submit(model_name,model,args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.load_state_dict(torch.load(model_name))
    model = model.to(device)
    test_samples = test_sample(data_path = args['LIBRI_PATH'], shuffle=False, partition="test-clean",csvpath='test_order.csv')
    test_acc = submit_test(args, model, device, test_samples)

    
    return model

In [ ]:
model = submit("/content/drive/MyDrive/11785/model/model_epoch_11_04_02_2022_18_52_28.pth",Network(args['context']),args)

In [ ]:
! kaggle competitions submit -c 11-785-s22-hw1p2 -f bad_04_02_2022_22_40_02.csv -m "Message"